In [1]:
from GoogleNews import GoogleNews
from newspaper import Article
from newsapi import NewsApiClient
import pandas as pd
import json
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import requests
import json
import csv
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import statsmodels as sm
import random
import os
import tweepy
from dotenv import load_dotenv
from pathlib import Path
import time
from deep_translator import GoogleTranslator
from googletrans import Translator
import warnings
warnings.filterwarnings('ignore')
from translate import Translator

In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import torch


# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []

    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task = 'sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"


tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels = []
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
#tokenizer.save_pretrained(MODEL)



In [3]:
def getSentiment(text):
    if text == None:
        return [0,1,0];
    #text = "Big Tech is getting wiped out in a brutal 3rd-quarter earnings season - but Apple isn't. Here's why."
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    # # TF
    # model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
    # model.save_pretrained(MODEL)

    # text = "Good night 😊"
    # encoded_input = tokenizer(text, return_tensors='tf')
    # output = model(encoded_input)
    # scores = output[0][0].numpy()
    # scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    sentiment = [0,0,0]
    index = {"negative": 0, "neutral": 1, "positive": 2}
    for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        sentiment[index[l]] = s
        #print(f"{i + 1}) {l} {np.round(float(s), 4)}")
    return sentiment

In [4]:
load_dotenv()

# OR, the same with increased verbosity
load_dotenv(verbose=True)

# OR, explicitly providing path to '.env'
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

# Print variable FOO
print(os.environ.get('API_KEY'))

None


In [5]:
client = tweepy.Client(os.environ.get("BEARER_TOKEN"))

In [6]:
df = pd.DataFrame()

In [18]:
tweets = None

def fetch_data_for_days(dir_name, begin, end, query):
    global df
    global tweets
    # period = timedelta(days=days_delta)
    six_hours = timedelta(hours=24)
    loop_time = timedelta(seconds=4)
    begin = datetime.strptime(begin, '%Y-%m-%d')
    end = datetime.strptime(end, '%Y-%m-%d')
    end_time = begin
    period = begin-end
    while(period > begin - end_time):
        print(end_time)
        start_loop_time = datetime.now()
        start_time = end_time - six_hours
        tweets = client.search_all_tweets(query=query, tweet_fields=['lang', 'context_annotations', 'created_at', 'possibly_sensitive', 'public_metrics'],
                                  start_time=start_time,
                                  end_time=end_time, sort_order="relevancy", max_results=30)
        end_time = start_time
        if(tweets.data==None):
            print("continuing because None dta")
            time.sleep(1)
            continue
            
        print(tweets)
        for tweet in tweets.data:
            if(tweet.lang!="en"):
                try:
                    tweet.text = GoogleTranslator(source=tweet.lang, target='en').translate(tweet.text)
                except:
                    continue
            sentiment = getSentiment(tweet.text)
            df = df.append({"id": tweet.id, 
                       "date": tweet.created_at , 
                       "text": tweet.text, 
                       "retweet_count": tweet.public_metrics["retweet_count"], 
                       "reply_count": tweet.public_metrics["reply_count"], 
                       "like_count": tweet.public_metrics["like_count"],
                       "quote_count": tweet.public_metrics["quote_count"],
                       "negative": sentiment[0],
                       "neutral": sentiment[1],
                       "positive": sentiment[2]
                      }, ignore_index=True)
        end_loop_time = datetime.now()
        diff = loop_time.total_seconds() - (end_loop_time-start_loop_time).total_seconds()
        if(diff>0):
            time.sleep(diff)
    file_name = begin.strftime("%Y") + " " + begin.strftime("%B") + ".csv"
    # df.to_csv("extracted_data_csi/"+file_name)
    df.to_csv(dir_name+file_name)
    return datetime.strftime(end_time, '%Y-%m-%d')

In [19]:
from dateutil.relativedelta import relativedelta
tweets = None
def fetch_data(dir_name, query, upto, forMonths):
    latest_date = upto
    # latest_date = "2022-10-31"
    latest_date = datetime.strptime(latest_date, '%Y-%m-%d')
    delta = relativedelta(months=1)
    # delta = relativedelta(days=1)
    for months in range(forMonths):
        first_date = latest_date - delta
        print(first_date)
        df = pd.DataFrame()
        latest_date = fetch_data_for_days(dir_name, latest_date.strftime('%Y-%m-%d'), first_date.strftime('%Y-%m-%d'), query)
        latest_date = datetime.strptime(latest_date, '%Y-%m-%d')


In [ ]:
tweets = None
df = pd.DataFrame()
fetch_data("extracted_data_csi", "#CSI300", "2020-02-01", 25)

In [22]:
tweets = None
df = pd.DataFrame()
fetch_data("extracted_data_s&p500", "s&p500", "2022-11-17", 60)

2022-10-17 00:00:00
2022-11-17 00:00:00
Response(data=[<Tweet id=1592790837054042112 text='S&amp;P500に投資をするな！\n→キャピタルしかないから云々…\n\nと、とあるYOUTUBERの方が言っていたが\n\nS&amp;P500のリターンの約半分はインカムによるものであり\n\nまたETFを使えば分配金という形で普通に（重要視しているであろう）インカムを貰うこともできるからまあ大丈夫だ👍\n\nhttps://t.co/YusfbmvfcX'>, <Tweet id=1592985368579612673 text='「ねぇねぇ、” つみたてNISA  ”ってどうやるの？」と親友に聞かれたので、三井住友カードつくって、SBI証券で口座開設して、AppleやAmazonを含むS&amp;P500を選んで、33,333円を毎月クレカ決済して、あとは20年間放置。年率5%だったら約800万円が約1,370万円になるよ。と教えてあげたら、翌日、親友から「'>, <Tweet id=1592796827228770306 text='(米国株)8年間株価低迷の罠から逃げよ！S&amp;P500,SOXL,レバナス,テスラ株(TSLA),ナスダック100,TECL,SPXLもFRBの黒歴史からチャンスを掴め！バフェット台湾TSMC投資も謎だ #SOXL #米国株 | お金が見る見る溶けていく――投資初心者がハマ... https://t.co/B9lc4dHpyK'>, <Tweet id=1592988881099444224 text='投資で大切にしていること\n\n・割安\n・EPS\n・配当\n・米国株\n・増配率\n・配当性向\n・配当再投資\n・競争優位性\n・Buy＆Hold\n・10−20銘柄\n・営業利益率\n・毎月一回の追加投資\n・25年以上の連続増配\n・売買ルールを絶対に守る\n・年初来でS＆P500を 12.82%超え\n・これが私の投資軸\n\n詳細は 固定ツイートです'>, <Tweet id=1592850035825606657 text='今日のeMAXIS Slim米国株式(S&amp;P500)は+53円。昨日のマイナスを取り戻す。

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
tweets = None
df = pd.DataFrame()
fetch_data("extracted_data_hsi", "#HSI", "2022-11-17", 60)

In [ ]:
tweets = None
df = pd.DataFrame()
fetch_data("extracted_data_nikkei225", "#nikkei225", "2022-11-17", 60)